In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler

def unet():
    inputs = Input((256,256,1))
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(2, 1, activation = 'sigmoid')(conv9)

    model = Model(input = inputs, output = conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

Using TensorFlow backend.


In [0]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import cv2
from os import listdir, makedirs
from os.path import isfile, join, exists

Path = '/content/gdrive/My Drive/Colab/Nails/data'
PathTrain = join(Path, 'train_two_classes')

def adjustData(img,mask):
    if(np.max(img) > 1):
        img = img / 255
        mask = np.delete(mask, np.s_[1], axis=3)
        mask = mask / 255
        mask[mask > 0.5] = 1
        mask[mask <= 0.5] = 0
    return (img,mask)

def trainGenerator(batch_size, train_path, image_folder, mask_folder, aug_dict, image_color_mode = "grayscale",
                    mask_color_mode = "grayscale", image_save_prefix  = "image", mask_save_prefix  = "mask",
                    save_to_dir = None, target_size = (256,256), seed = 1):

    image_datagen = ImageDataGenerator(**aug_dict)
    mask_datagen = ImageDataGenerator(**aug_dict)
    image_generator = image_datagen.flow_from_directory(
        train_path,
        classes = [image_folder],
        class_mode = None,
        color_mode = image_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = image_save_prefix,
        seed = seed)
    mask_generator = mask_datagen.flow_from_directory(
        train_path,
        classes = mask_folder,
        class_mode = None,
        color_mode = 'rgb',
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = mask_save_prefix,
        seed = seed)
    train_generator = zip(image_generator, mask_generator)
    for (img,mask) in train_generator:
        img,mask = adjustData(img,mask)
        yield (img,mask)

def testData(test_path, target_size = (256,256)):
  onlyfiles = [f for f in listdir(test_path) if isfile(join(test_path, f))]
  result = []
  file_names = []
  for f in onlyfiles:
    file_names.append(f)
    img = cv2.imread(join(test_path, f), cv2.IMREAD_GRAYSCALE)
    img = img / 255
    img = cv2.resize(img, target_size, interpolation = cv2.INTER_AREA)
    img = np.reshape(img, (1,) + img.shape + (1,))
    result.append(img)
  return result, file_names

def resize_coordinates(coordinates,old_size,new_size):
  if len(coordinates) == 0:
    return coordinates
  coordinates[:,0] = coordinates[:,0] * (new_size[1]/old_size[1])
  coordinates[:,1] = coordinates[:,1] * (new_size[0]/old_size[0])
  return coordinates

def find_centers(mask, minimal_size_prsent = 1):
  contours=find_contours(mask)
  minimal_size = mask.shape[0] * mask.shape[1] * (minimal_size_prsent/100)
  return find_centers_by_contours(contours, minimal_size)

def find_contours(mask):
  m1 = mask.astype(np.uint8).copy()
  im2, contours, hierarchy = cv2.findContours(m1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
  return contours
  
def find_centers_by_contours(contours, minimal_size):
  centers = []
  areas = []
  cont = []
  for cnt in contours:
    M = cv2.moments(cnt)
    if M["m00"] != 0:      
      if M["m00"] > minimal_size:
        areas.append(M["m00"])
        cont.append(np.array(cnt))
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        centers.append([cX,cY])  
  return np.array(centers), np.array(areas), cont


def normalize(mask):
  mask = mask[0,:,:,:]
  if mask[:,:,0].max() > 0:
    mask[:,:,0] = (mask[:,:,0]/mask[:,:,0].max())
  if mask[:,:,1].max() > 0:
    mask[:,:,1] = (mask[:,:,1]/mask[:,:,1].max())
  
  mask[mask > 0.5] = 1
  mask[mask <= 0.5] = 0
  mask=mask*255
  return mask

def get_common_center(cnt, conturs):
  common_border = []
  for cc in conturs:
    mutch = False
    for p in cnt:
      for c in cc:
        if p[0][0] == c[0][0] and p[0][1] == c[0][1]:
          mutch = True
    if mutch:
      common_border.append(cc)
  
  centers0, areas0, cont0 = find_centers_by_contours(common_border, 0.05)
  if len(areas0) >0:
    i = np.argmax(areas0)
    return centers0[i]
  else:
    return []

def get_data(mask):
  cont0 = find_contours(mask[:,:,0])
  centers1, areas1, cont1 = find_centers(mask[:,:,1], 0.3)
  centers_new = []
  for cont in cont1:
      common_center = get_common_center(cont, cont0)
      centers_new.append(common_center)
  
  return list(zip(centers1, centers_new))

  
def saveResult(npMask, save_path, fileName, original_path, train_path = ''):
  if not exists(join(PathTrain, save_path)):
    makedirs(join(PathTrain, save_path))
  
  mask = normalize(npMask)
  
  img_original = cv2.imread(join(original_path, fileName))
  mask = cv2.resize(mask, (img_original.shape[1], img_original.shape[0])).astype(int)
  
  centers = get_data(mask)
  
  img_original[:,:,0] = (img_original[:,:,0]).astype(int)
  img_original[:,:,1] = (img_original[:,:,1] / (1 + (mask[:,:,1] / 192))).astype(int)
  img_original[:,:,2] = (img_original[:,:,2] / (1 + (mask[:,:,0] / 192))).astype(int)

  for c in centers:
    cv2.circle(img_original, tuple(c[0]), 3, (255, 0, 0))
    if len(c[1])>0:
      cv2.circle(img_original, tuple(c[1]), 3, (255, 255, 255))
      cv2.circle(img_original, tuple(c[1]), 4, (0, 0, 0))
      cv2.line(img_original,tuple(c[0]),tuple(c[1]),(255,255,255),2)
  
  if train_path != '':
    train_files = [f for f in listdir(train_path) if isfile(join(train_path, f))]
    if fileName in train_files:
      cv2.putText(img_original,'train',(20,20),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)
  cv2.imwrite(join(join(PathTrain, save_path),fileName),img_original)

In [0]:

from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras.backend import clear_session
def train(checkpoint_to_save, steps_per_epoch, epochs, checkpoint_to_load = ''):
  clear_session()
  data_gen_args = dict(rotation_range=0.6,
                      width_shift_range=0.1,
                      height_shift_range=0.1,
                      shear_range=0.1,
                      zoom_range=0.1,
                      horizontal_flip=True,
                      fill_mode='nearest')

  myGene = trainGenerator(100, PathTrain, 'pics', ['masks'], data_gen_args)
  model = unet()
  if checkpoint_to_load != '':
    model.load_weights(join(PathTrain, checkpoint_to_save))
  model_checkpoint = ModelCheckpoint(join(PathTrain, checkpoint_to_save), monitor='loss',verbose=1, save_best_only=True)
  model.fit_generator(myGene,steps_per_epoch,epochs,callbacks=[model_checkpoint])

In [0]:
from datetime import datetime
def test(checkpoint, results_path):
  X, file_names = testData(Path + '/test')
  model = unet()
  model.load_weights(join(PathTrain, checkpoint))
  for file_name, x in zip(file_names, X):
    print(file_name)
    t1 = datetime.now()
    result = model.predict(x, verbose=0)
    t2 = datetime.now()    
    print(t2-t1)
    saveResult(result, results_path, file_name, Path + '/test', PathTrain + '/pics')
    print(datetime.now()-t2)

In [6]:
#train('unet_two_classes1.hdf5', 100, 1)
#test('unet_two_classes.hdf5', 'test_results_two_classes')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`


d61ac71a-db67-11e8-9658-0242ac1c0002.jpg


KeyboardInterrupt: ignored

In [8]:
for i in range(100):
  print('step ' + str(i))
  train('unet_two_classes.hdf5', 300, 1, 'unet_two_classes.hdf5')
  #test('unet_two_classes.hdf5', 'test_results_two_classes')

step 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`


Epoch 1/1
Found 22 images belonging to 1 classes.
Found 22 images belonging to 1 classes.
138/300 [============>.................] - ETA: 9:06 - loss: 0.0016 - acc: 0.9994

KeyboardInterrupt: ignored